**Notebook 1: Setting the Scene**¡

This notebook sets up a supply chain optimization model using Pyomo and the CBC solver to simulate and stress-test a simple supply network consisting of suppliers, factories, and customers. The core idea is to model how products flow through the network—accounting for costs, capacity constraints, and potential disruptions like factory outages—and then optimize operations to minimize total cost. It includes logic for handling surge capacity (extra capacity at factories), penalties for unmet customer demand, and a scenario runner that logs results to MLflow for tracking. The simulations include a baseline (everything working fine) and two stress scenarios where one factory is taken offline to observe how the network adapts and what the cost and unmet demand implications are. Think of it as a mini digital twin of a supply chain, built to poke it with a stick and see where it breaks.


Use this HW configuration to test it

**Cluster:**
- Single-node 
- Runtime: Databricks Runtime with ML (15.4 LTS)

- Driver node:
> - AWS: (Tried first with ) i3.2xlarge
> - Azure: (Might be enough) Standard_DS4_v3
- Workers: 0  

Why: We’re solving small LP/MIP problems. No need to overprovision. 

**Libraries:**
- pyomo (via %pip install pyomo)
- CBC installed via init script ( you can find it  in the same directory as this notebook) 



In [0]:
%run "./init"

In [0]:
def run_scenario(scenario_name, factory_down=None, ttr=0):
    cap = factory_capacity.copy()
    extra = extra_capacity.copy()
    
    if factory_down:
        cap[factory_down] = 0
        extra[factory_down] = 0
    
    model = build_model(cap, extra)
    solver = pyo.SolverFactory("cbc")
    result = solver.solve(model)

    cost = pyo.value(model.obj)
    unmet = sum(model.unmet[c].value for c in model.C)
    extra_used = {f: model.extra_used[f].value for f in model.F}

    mlflow.log_param("scenario", scenario_name)
    mlflow.log_param("factory_down", factory_down or "None")
    mlflow.log_metric("total_cost", cost)
    mlflow.log_metric("total_unmet_demand", unmet)
    for f, v in extra_used.items():
        mlflow.log_metric(f"extra_used_{f}", v)

    return {
        "scenario": scenario_name,
        "cost": cost,
        "unmet": unmet,
        "extra_used": extra_used
    }

In [0]:

experiment = mlflow.get_experiment_by_name(experiment_name)

if experiment:
    mlflow.delete_experiment(experiment.experiment_id)

mlflow.set_experiment(experiment_name)

with mlflow.start_run(run_name="Baseline"):
    baseline_results = run_scenario("Baseline")

with mlflow.start_run(run_name="F1_Down"):
    f1_results = run_scenario("F1_Down", factory_down="F1")

with mlflow.start_run(run_name="F2_Down"):
    f2_results = run_scenario("F2_Down", factory_down="F2")